In [15]:
from __future__ import division
import numpy as np
from PIL import Image
import openslide as opsl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LogNorm
import progressbar
from sklearn.cluster import KMeans
from sklearn.externals import joblib
from sklearn import neighbors
import time
import cv2
import gc
import os,sys
import random
sys.path.append('/cptjack/totem/StainTools/')
from utils import visual_utils as vu
from utils import misc_utils as mu
from normalization.vahadane import VahadaneNormalizer

In [2]:
def img_color_hist(folder_zero,bin_count):
    dir_img = os.listdir(folder_zero+'/')
#    file_num = sum([len(files) for root, dirs, files in os.walk(folder_zero)])
    data = np.zeros((len(dir_img),bin_count,1,3))
    p = progressbar.ProgressBar()
    for i in p(range(len(dir_img))):
        Img = cv2.imread(folder_zero+'/'+dir_img[i])
        for j in range(3):
            data[i,:,:,j] = cv2.calcHist([Img],[j],None,[bin_count],[0,256])
    data = data.reshape(len(dir_img),bin_count*3)
    return data

In [21]:
def img_predict(folder_zero,bin_count,kmean_model,save_dir):
    dir_img = dir_img = os.listdir(folder_zero+'/')
    p = progressbar.ProgressBar()
    for i in p(range(len(dir_img))):
        data_test = np.zeros((1,bin_count,1,3))
        Img_test = cv2.imread(folder_zero+'/'+dir_img[i])
        for j in range(3):
            data_test[0,:,:,j] = cv2.calcHist([Img_test],[j],None,[bin_count],[0,256])
        data_test = data_test.reshape(1,bin_count*3)
        label = kmean_model.predict(data_test)
        if label==0:
            sub_dir='0'
        else:
            sub_dir='1'
        cv2.imwrite(save_dir+'/'+sub_dir+'/'+dir_img[i],Img_test)

In [23]:
if __name__ == '__main__':    
    dir_zero = '/cptjack/totem/18738_in'
    data = img_color_hist(dir_zero,128)
    

100% (4708 of 4708) |####################| Elapsed Time: 0:00:43 Time:  0:00:43


In [4]:
    data.shape

(4708, 192)

In [24]:
    if os.path.exists('/cptjack/totem/Colon Pathology/kmeans_model_128.m'):
        k = joblib.load('/cptjack/totem/Colon Pathology/kmeans_model_128.m')
    else:
        k = KMeans(n_clusters=2,random_state=0,max_iter=4000,tol=0.00001,algorithm='full').fit(data)
        joblib.dump(k,'/cptjack/totem/Colon Pathology/kmeans_model_128.m')    

In [25]:
    save_result_dir = '/cptjack/totem/18738_k_meas_test_128'
    if not os.path.exists(save_result_dir):os.makedirs(save_result_dir)  
    img_predict(dir_zero,128,k,save_result_dir)

100% (4708 of 4708) |####################| Elapsed Time: 0:00:16 Time:  0:00:16


In [11]:
    dir_img = dir_img = os.listdir(dir_zero+'/')
    data_test = np.zeros((1,64,1,3))
    print(dir_img[1])
    Img_test = cv2.imread(dir_zero+'/'+dir_img[1])
    for j in range(3):
        data_test[0,:,:,j] = cv2.calcHist([Img_test],[j],None,[64],[0,256])
    data_test = data_test.reshape(1,64*3)
    label = k.predict(data_test)
    print(label)

18738__y11295_x17851_p1.png
[0]


In [14]:
    if label==0:
        print('done!')

done!
